In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.memory import ConversationBufferWindowMemory

#### NOTE: the "k" value of only one.  That will roll older messages out of memory and only keep the most recent one.

In [12]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [13]:
# return_messages has the same effect here as in ConversationalBuffer
memory = ConversationBufferWindowMemory(k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you'),
  AIMessage(content='not much')]}

#### Using in a chain

In [22]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="My name is Dan")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Dan
AI:

> Finished chain.


" Hi Dan, my name is AI. It's nice to meet you. What can I do for you today?"

In [25]:
conversation_with_summary.predict(input="Could you tell me a joke?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Do you remember by name?
AI:  Yes, I remember your name is Dan. I have stored it in my memory. Is there anything else I can help you with?
Human: That's funny how about another one?
AI:  Sure, what would you like to know?
Human: Could you tell me a joke?
AI:

> Finished chain.


" I'm sorry, I don't know any jokes. However, I can tell you a funny story. Would you like to hear one?"

In [24]:
conversation_with_summary.predict(input="That's funny how about another one?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Dan
AI:  Hi Dan, my name is AI. It's nice to meet you. What can I do for you today?
Human: Do you remember by name?
AI:  Yes, I remember your name is Dan. I have stored it in my memory. Is there anything else I can help you with?
Human: That's funny how about another one?
AI:

> Finished chain.


' Sure, what would you like to know?'

In [26]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="Do you remember by name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: That's funny how about another one?
AI:  Sure, what would you like to know?
Human: Could you tell me a joke?
AI:  I'm sorry, I don't know any jokes. However, I can tell you a funny story. Would you like to hear one?
Human: Do you remember by name?
AI:

> Finished chain.


" I'm sorry, I don't understand the question. Could you please rephrase it?"